In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The code cell below fetches the `global_air_quality` table from the `openaq` dataset.  We also preview the first five rows of the table.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [15]:
df = client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


In [16]:
df.head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


# Exercises

### 1) Units of measurement

Which countries have reported pollution levels in units of "ppm"?  In the code cell below, set `first_query` to an SQL query that pulls the appropriate entries from the `country` column.

In case it's useful to see an example query, here's some code from the tutorial:

```
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
```

In [17]:
# Query to select countries with units of "ppm"
first_query = """
                SELECT DISTINCT country
                FROM `bigquery-public-data.openaq.global_air_quality`
                WHERE unit='ppm'
"""

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


  country
0      US
1      CL
2      AU
3      BM
4      MX


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution

In [18]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

first_query = """
              SELECT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """

# Or to get each country just once, you could use
first_query = """
              SELECT DISTINCT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """

```

### 2) High air quality

Which pollution levels were reported to be exactly 0?  
- Set `zero_pollution_query` to select **all columns** of the rows where the `value` column is 0.
- Set `zero_pollution_results` to a pandas DataFrame containing the query results.

In [19]:
# Query to select all columns where pollution levels are exactly 0
zero_pollution_query = """
                SELECT *
                FROM `bigquery-public-data.openaq.global_air_quality`
                WHERE value=0
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

# API request - run the query and return a pandas DataFrame
zero_pollution_results = query_job.to_dataframe()

print(zero_pollution_results.head())

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


                                        location     city country pollutant  \
0                     Victoria Memorial - WBSPCB  Kolkata      IN      pm25   
1  Rabindra Bharati University, Kolkata - WBSPCB  Kolkata      IN       so2   
2                   Zamość ul. Hrubieszowska 69A   Zamość      PL       no2   
3                               Końskie, MOBILNA  Końskie      PL      pm10   
4                               Końskie, MOBILNA  Końskie      PL      pm25   

   value                 timestamp   unit source_name   latitude  longitude  \
0    0.0 2017-10-16 20:45:00+00:00  µg/m³        CPCB  22.572645  88.363890   
1    0.0 2017-10-28 14:30:00+00:00  µg/m³        CPCB  22.627874  88.380400   
2    0.0 2020-05-19 05:00:00+00:00  µg/m³        GIOS  50.716630  23.290247   
3    0.0 2018-12-21 13:00:00+00:00  µg/m³        GIOS  51.189526  20.408892   
4    0.0 2018-12-21 13:00:00+00:00  µg/m³        GIOS  51.189526  20.408892   

   averaged_over_in_hours  
0                    0

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution

In [21]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

zero_pollution_query = """
                       SELECT *
                       FROM `bigquery-public-data.openaq.global_air_quality`
                       WHERE value = 0
                       """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

zero_pollution_results = query_job.to_dataframe()

```

That query wasn't too complicated, and it got the data you want. But these **SELECT** queries don't organize data in a way that answers the most interesting questions. For that, we'll need the **GROUP BY** command. 

If you know how to use [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) in pandas, this is similar. But BigQuery works quickly with far larger datasets.

Fortunately, that's next.

## Let's use Folium and filter for a subset of the records
Source: https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [22]:
df = client.list_rows(table, max_results=10000).to_dataframe()
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.000000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.870000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.570000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.620000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.490000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
...,...,...,...,...,...,...,...,...,...,...,...
9995,IT1646A,Lodi,IT,no2,39.731090,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.081993,9.713890,1.00
9996,IT0692A,Pavia,IT,no2,15.283235,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.101670,8.905560,1.00
9997,IT0692A,Pavia,IT,so2,2.640760,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.101670,8.905560,1.00
9998,IT2072A,Rovigo,IT,no2,14.000000,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.103890,11.553333,1.00


In [23]:
df = df[df.pollutant == 'no2'].reset_index()
df

,index,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.870000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
1,6,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,no2,166.550000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
2,11,City Railway Station - KSPCB,Bengaluru,IN,no2,46.860000,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.977347,77.570694,0.25
3,15,SaneguravaHalli - KSPCB,Bengaluru,IN,no2,23.670000,2018-02-21 23:45:00+00:00,µg/m³,CPCB,12.991670,77.545830,0.25
4,19,IIT,Chennai,IN,no2,6.130000,2018-02-22 03:45:00+00:00,µg/m³,CPCB,12.992514,80.237450,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
1780,9990,IT1215A,Rovigo,IT,no2,24.000000,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.074720,11.783330,1.00
1781,9993,IT0469A,Torino,IT,no2,27.000000,2020-06-10 23:00:00+00:00,µg/m³,EEA Italy,45.075850,7.678220,1.00
1782,9995,IT1646A,Lodi,IT,no2,39.731090,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.081993,9.713890,1.00
1783,9996,IT0692A,Pavia,IT,no2,15.283235,2020-06-10 21:00:00+00:00,µg/m³,EEA Italy,45.101670,8.905560,1.00


In [24]:
import folium

In [25]:
locations = df[['latitude', 'longitude']].dropna()
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[19.041847, 72.86552]

In [26]:
folium_colors = ['green', 'orange', 'red', 'darkred']

In [27]:
df_value_mean = df['value'].mean()
df_value_mean

-11.287871119327733

In [28]:
quantile_25 = df['value'].quantile(0.25)
quantile_50 = df['value'].quantile(0.50)
quantile_75 = df['value'].quantile(0.75)
quantiles = [0 , quantile_25, quantile_50, quantile_75]
quantiles

[0, 0.007, 5.18671, 15.0]

In [29]:
import numpy as np

In [30]:
df.value[:5]

0    131.87
1    166.55
2     46.86
3     23.67
4      6.13
Name: value, dtype: float64

In [31]:
np.digitize([0, 14, 43, 211], quantiles)

array([1, 3, 4, 4])

In [32]:
df.head()

,index,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.609220,0.25
1,6,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,no2,166.55,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.697270,0.25
2,11,City Railway Station - KSPCB,Bengaluru,IN,no2,46.86,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.977347,77.570694,0.25
3,15,SaneguravaHalli - KSPCB,Bengaluru,IN,no2,23.67,2018-02-21 23:45:00+00:00,µg/m³,CPCB,12.991670,77.545830,0.25
4,19,IIT,Chennai,IN,no2,6.13,2018-02-22 03:45:00+00:00,µg/m³,CPCB,12.992514,80.237450,0.25


In [33]:
map = folium.Map(location=locationlist[7], zoom_start=2)

for point in range(0, len(locationlist)):
    text = df['value'][point]
    city = df['city'][point]
    country = df['country'][point]
    pollutant = df['pollutant'][point]
    color_index = np.digitize(text, quantiles) - 1
    #print(text, color_index, city)
    color = folium_colors[color_index]
    folium.Marker(locationlist[point], popup=(city, country, pollutant, text), icon=folium.Icon(color=color)).add_to(map)

In [34]:
map